# Batched Cgauss 

In [1]:
import numpy as np
import torch as th

import time
import cProfile

In [508]:
dtype = th.float64

gpuid = 0
device = th.device("cuda:"+ str(gpuid))
#device = th.device("cpu")

print("Execution device: ",device)
print("PyTorch version: ", th.__version__ )
print("CUDA available: ", th.cuda.is_available())
print("CUDA version: ", th.version.cuda)
print("CUDA device:", th.cuda.get_device_name(gpuid))

Execution device:  cuda:0
PyTorch version:  0.4.1
CUDA available:  True
CUDA version:  9.0.176
CUDA device: TITAN V


In [509]:
def vech2L(v,n):
    count = 0
    L = th.zeros((n,n))
    for j in range(n):
        for i in range(j,n):
            L[i,j]=v[count]
            count = count + 1
    return th.tensor(L , device=device, dtype=dtype)

# batched vech2L input is "X" as V nb x n(n+1)/2
def bvech2L(V,nb,n):
    count = 0
    L = th.zeros((nb,n,n))
    for j in range(n):
        for i in range(j,n):
            L[...,i,j]=V[...,count]
            count = count + 1
    return th.tensor(L , device=device, dtype=dtype)

def cholesky(A):
    L = th.zeros_like(A)
    
    for i in range(A.shape[-1]):
        for j in range(i+1):
            s = 0.0
            for k in range(j):
                s = s + L[...,i,k].clone() * L[...,j,k].clone()
            
            L[...,i,j] = th.sqrt(A[...,i,i] - s) if (i == j) else \
                      (1.0 / L[...,j,j].clone() * (A[...,i,j] - s))
    return L

def inverseL(L):
    n = L.shape[-1]
    invL = th.zeros_like(L)
    for j in range(0,n):
        invL[...,j,j] = 1.0/L[...,j,j]
        for i in range(j+1,n):
            S = 0.0
            for k in range(i+1):
                S = S - L[...,i,k]*invL[...,k,j].clone()
            invL[...,i,j] = S/L[...,i,i]

    return invL
            

In [510]:
def b_matel(n, Ak, Al, invAkl, detAkl, detLk, detLl, Mass, Qmat, Rkl):
    
    
    # apply symmetry projection on Ll
    
    # th.t() is shorthand for th.transpose(X, 0,1)
    #PLl = th.t(Sym) @ Ll;
    
    # build Ak, Al, Akl, invAkl, invAk, invAl

    #Ak = Lk@th.t(Lk);
    #Al = PLl@th.t(PLl);
    #Akl = Ak+Al;
    #print(Al)
    #invAkl = th.inverse(Akl);
    # Overlap: (normalized)
    skl = 2**(n*1.5) * th.sqrt( th.pow(detLk*detLl/detAkl ,3) );

    # kinetic energy
    #print('Ak',Ak)
    #print('Al',Al)
    #print('invAkl', invAkl)
    #print('Ak invAkl Al', Ak@invAkl@Al)
    tkl = skl*(6*th.trace(Mass@Ak@invAkl@Al));    
    #tkl = skl*(6*th.sum(Mass*(Ak@invAkl@Al)))
    
    # potential energy
    
    TWOoSqrtPI = 1.1283791670955126 # 2/sqrt(pi)
    
    #RIJ = th.zeros((n,n), device=device, dtype=dtype);
    # 1/rij i~=j
    #for j in range(0,n-1):
    #    for i in range(j+1,n):
    #        tmp2 = invAkl[i,i] + invAkl[j,j] - 2*invAkl[i,j];
    #        #RIJ[i,j] = TWOoSqrtPI * skl/th.sqrt(tmp2);
    #        RIJ[i,j] = th.rsqrt(tmp2)


    # 1/rij i=j
    #for i in range(0,n):
        #RIJ[i,i] = TWOoSqrtPI * skl/th.sqrt(invAkl[i,i]);
    #    RIJ[i,i] = th.rsqrt(invAkl[i,i])
    
    Rkl = TWOoSqrtPI*skl*Rkl
    
    #Q = vech2L(vecQ,n);

    vkl = th.sum(Rkl*Qmat)

    #hkl = tkl + vkl
    
    
    return {'skl':skl, 'tkl':tkl, 'vkl':vkl}


In [511]:
def test_matel():
    n = 3;
    nb = 2
    nn = int(n*(n+1)/2)
    
    vechLk = th.tensor([  1.00000039208682, 
              0.02548044275764261, 
              0.3525161612610669,
              1.6669144815242515,
              0.9630555318946559,
              1.8382882034659822 ], device=device, dtype=dtype, requires_grad=True);
    
    vechLl = th.tensor([  1.3353550436464964,
               0.9153272033682132,
               0.7958636766525028,
               1.8326931436447955,
               0.3450426931160630,
               1.8711839323167831 ], device=device, dtype=dtype, requires_grad=True);
    
    Sym = th.tensor([[0,0,1],
                    [0,1,0],
                    [1,0,0]], device=device, dtype=dtype);
    
    #Sym = th.tensor([[1,0,0],
     #               [0,1,0],
      #              [0,0,1]], device=device, dtype=dtype);
    
    Mass = th.tensor([[5.446170e-4, 2.723085077e-4, 2.723085077e-4],
                     [2.723085077e-4, .5002723085, 2.723085077e-4],
                     [2.723085077e-4, 2.723085077e-4, .5002723085 ]], device=device, dtype=dtype);
    
    Qmat = vech2L(th.tensor([1, -1, -1, -1, 1, -1], device=device, dtype=dtype),n);
    
    x = th.cat((vechLk,vechLl))
    # reshape non-linear variables for easier indexing
    X = th.reshape(x[:nb*nn], (nb,nn))
    #npX = X.detach().numpy()
    L = th.zeros((nb,n,n), device=device, dtype=dtype)
    #for i in range(nb):
    #    L[i][:,:] = vech2L(X[i,:],n)
    L = bvech2L(X,nb,n)

    detL = th.abs(th.prod(th.diagonal(L, offset=0, dim1=-1, dim2=-2),1))
    AK = th.matmul(L,th.transpose(L, 1, 2))
    #PL = th.matmul(th.t(Sym),L)
    #AL = th.matmul(PL,th.transpose(PL, 1, 2))
    AL = th.matmul(th.t(Sym), th.matmul(AK,Sym))
    
    AKL = th.zeros((nb,nb,n,n), device=device, dtype=dtype)
    for i in range(nb):
        for j in range(nb):
            AKL[i,j] =  th.add(AK[i], AL[j]) 
    cholAKL = cholesky(AKL)
    detAKL = th.prod(th.diagonal(th.reshape(cholAKL, (nb*nb,n,n)), offset=0, dim1=-1, dim2=-2),1)**2
    #print(detAKL[0])
    #detAKL *= detAKL
    detAKL = th.reshape(detAKL, (nb,nb))
    invLKL=inverseL(cholAKL)
    #print(invLKL)
    #print(th.transpose(invLKL, dim0=-2, dim1=-1))
    invAKL = th.matmul(th.transpose(invLKL, dim0=-1, dim1=-2),invLKL)
    #print(th.prod(th.diag(cholAKL[0,0])))
    #print(th.prod(th.diag(th.potrf(AKL[0,0],upper=False))))
    #print(th.prod(th.diagonal(cholAKL[0,0], offset=0, dim1=-1, dim2=-2),0))
    
    RIJ = th.zeros_like(invAKL, device=device, dtype=dtype);
    # 1/rij i~=j
    for j in range(0,n-1):
        for i in range(j+1,n):
            tmp2 = invAKL[...,i,i] + invAKL[...,j,j] - 2*invAKL[...,i,j];
            #RIJ[i,j] = TWOoSqrtPI * skl/th.sqrt(tmp2);
            RIJ[...,i,j] = th.rsqrt(tmp2)

    # 1/rij i=j
    for i in range(0,n):
        #RIJ[i,i] = TWOoSqrtPI * skl/th.sqrt(invAkl[i,i]);
        RIJ[...,i,i] = th.rsqrt(invAKL[...,i,i])    
    
    # Overlap: (normalized)
    #print(th.ger(detL, detL)/detAKL)
    SKL = 2**(n*1.5) * th.sqrt( th.pow(th.ger(detL, detL)/detAKL ,3) );
    #print(SKL)
    
    # kinetic energy
    #TKL= SKL*6*th.sum(Mass*th.matmul(AK, th.matmul(invAKL,AL)),(2,3))
    
    C = th.zeros_like(invAKL)
    for i in range(nb):
        for j in range(nb):
            C[i,j] = (AK[i]@invAKL[i,j]@AL[j])
    #C1 = th.matmul(invAKL,AL)
    #C = th.matmul(AK, C1)
    #C = th.matmul(invAKL,AL)
    #print('bAK',AK)
    #print('bAL',AL)
    #print('binvAKL', invAKL)
    #print('bAK invAKL AL', C)
    #CT = th.transpose(C, dim0=-1, dim1=-2)
    #TKL = 6*SKL*th.sum(th.diagonal(C, dim1=-2, dim2=-1), dim=-1, keepdim=False)
    TKL = 6*SKL*th.sum(Mass*C, dim=(-2,-1))
    
    print(TKL)
    #TKL = SKL*(6*th.trace(Mass@Ak@invAkl@Al))    
    #tkl = skl*(6*th.sum(Mass*(Ak@invAkl@Al)))
    
    # potential energy
    TWOoSqrtPI = 1.1283791670955126 # 2/sqrt(pi)
    
    VKL = TWOoSqrtPI*SKL*th.sum(RIJ*Qmat,dim=(-2,-1))
    #print(VKL)

    
    for i in range(0,nb):
        for j in range(i,nb):
            Ai = AK[i,:,:]
            Aj = AL[j,:,:]
            Aij = AKL[i,j]
            #print(Aj)
            detLi = detL[i]
            detLj = detL[j]
            detAij = detAKL[i,j]
            #detAij = th.det(Ai+Aj)
            #print(detAij)
            #print(th.det(Ai+Aj))
            #print(Aij)
            #print(Ai+Aj)
            invAij = invAKL[i,j]
            #print(i,j)
            Rij = RIJ[i,j]
            matels = b_matel(n, Ai, Aj, invAij, detAij, detLi, detLj, Mass, Qmat,Rij)
    
            print('skl: ',matels['skl'])
            print('tkl: ',matels['tkl'])
            print('vkl: ',matels['vkl'])
    
    #tkl = matels['tkl']
    #print(th.autograd.grad(tkl, vechLk, retain_graph=True) ) 

In [512]:
test_matel()

tensor([[2.3546, 4.3509],
        [3.7564, 6.7999]], device='cuda:0', dtype=torch.float64, grad_fn=<ThMulBackward>)
skl:  tensor(0.4115, device='cuda:0', dtype=torch.float64, grad_fn=<MulBackward>)
tkl:  tensor(2.3546, device='cuda:0', dtype=torch.float64, grad_fn=<ThMulBackward>)
vkl:  tensor(-1.7185, device='cuda:0', dtype=torch.float64, grad_fn=<SumBackward0>)
skl:  tensor(0.5334, device='cuda:0', dtype=torch.float64, grad_fn=<MulBackward>)
tkl:  tensor(4.3509, device='cuda:0', dtype=torch.float64, grad_fn=<ThMulBackward>)
vkl:  tensor(-2.3840, device='cuda:0', dtype=torch.float64, grad_fn=<SumBackward0>)
skl:  tensor(0.6856, device='cuda:0', dtype=torch.float64, grad_fn=<MulBackward>)
tkl:  tensor(6.7999, device='cuda:0', dtype=torch.float64, grad_fn=<ThMulBackward>)
vkl:  tensor(-3.0929, device='cuda:0', dtype=torch.float64, grad_fn=<SumBackward0>)


In [521]:
def b_energyrc(x,n,nb,Mass,Qmat,Sym,symc):
    
    nx = len(x);
    nn = int(n*(n+1)/2);
    nsym = len(symc);
    #nsym = 1
    
    # extract linear coefs "eigen vector"
    c = x[-nb:];
    # reshape non-linear variables for easier indexing
    X = th.reshape(x[:nb*nn], (nb,nn))
    #npX = X.detach().numpy()
    L = th.zeros((nb,n,n), device=device, dtype=dtype)

    L = bvech2L(X,nb,n)

    detL = th.abs(th.prod(th.diagonal(L, offset=0, dim1=-1, dim2=-2),1))
    AK = th.matmul(L,th.transpose(L, 1, 2))

    
    # Build H and S
    H = th.zeros((nb,nb), device=device, dtype=dtype);
    S = th.zeros((nb,nb), device=device, dtype=dtype);
    T = th.zeros((nb,nb), device=device, dtype=dtype);
    V = th.zeros((nb,nb), device=device, dtype=dtype);
    
    # Build H and S
    bH = th.zeros((nb,nb), device=device, dtype=dtype);
    bS = th.zeros((nb,nb), device=device, dtype=dtype);
    bT = th.zeros((nb,nb), device=device, dtype=dtype);
    bV = th.zeros((nb,nb), device=device, dtype=dtype);
    
    # outer loop is over symmetry terms
    for k in range(0,nsym):
        
        P = Sym[:,:,k]

        AL = th.matmul(th.t(P), th.matmul(AK,P))

        AKL = th.zeros((nb,nb,n,n), device=device, dtype=dtype)
        for i in range(nb):
            for j in range(nb):
                AKL[i,j] =  th.add(AK[i], AL[j]) 
        cholAKL = cholesky(AKL)
        #cholAKL = th.zeros_like(AKL)
        #for i in range(nb):
        #    for j in range(nb):
        #        cholAKL[i,j] = th.potrf(AKL[i,j], upper=False) 
        
        detAKL = th.prod(th.diagonal(cholAKL, offset=0, dim1=-1, dim2=-2),-1)**2
        #print(detAKL[0])
        #detAKL *= detAKL
        #detAKL = th.reshape(detAKL, (nb,nb))
        invLKL = inverseL(cholAKL)
        #invLKL = th.zeros_like(AKL)
        #for i in range(nb):
        #    for j in range(nb):
        #        invLKL[i,j] = th.inverse(cholAKL[i,j]) 


        invAKL = th.matmul(th.transpose(invLKL, dim0=-1, dim1=-2),invLKL)

        RIJ = th.zeros_like(invAKL, device=device, dtype=dtype);
        # 1/rij i~=j
        for j in range(0,n-1):
            for i in range(j+1,n):
                tmp2 = invAKL[...,i,i] + invAKL[...,j,j] - 2*invAKL[...,i,j];
                #RIJ[i,j] = TWOoSqrtPI * skl/th.sqrt(tmp2);
                RIJ[...,i,j] = th.rsqrt(tmp2)

        # 1/rij i=j
        for i in range(0,n):
            #RIJ[i,i] = TWOoSqrtPI * skl/th.sqrt(invAkl[i,i]);
            RIJ[...,i,i] = th.rsqrt(invAKL[...,i,i])    


        # Overlap: (normalized)
        #print(th.ger(detL, detL)/detAKL)
        SKL = 2**(n*1.5) * th.sqrt( th.pow(th.ger(detL, detL)/detAKL ,3) );
        #print(symc[k]*SKL)

        # kinetic energy
        C = th.zeros_like(invAKL)
        for i in range(nb):
            for j in range(nb):
                C[i,j] = (AK[i]@invAKL[i,j]@AL[j])
        #C = th.matmul(AK, th.matmul(invAKL,AL))
        #TKL = 6*SKL*th.sum(th.diagonal(C, dim1=-2, dim2=-1), dim=-1, keepdim=False)
        TKL = 6*SKL*th.sum(Mass*C, dim=(-2,-1))
        #print(symc[k]*TKL)
        #TKL = SKL*(6*th.trace(Mass@Ak@invAkl@Al))    
        #tkl = skl*(6*th.sum(Mass*(Ak@invAkl@Al)))

        # potential energy
        TWOoSqrtPI = 1.1283791670955126 # 2/sqrt(pi)
        
        VKL = TWOoSqrtPI*SKL*th.sum(RIJ*Qmat, dim=(-2,-1))
        #print(symc[k]*VKL)
    
            
        #for i in range(0,nb):
        #    for j in range(i,nb):
        #        Ai = AK[i,:,:]
        #        Aj = AL[j,:,:]
        #        Aij = AKL[i,j]
        #        detLi = detL[i]
        #        detLj = detL[j]
        #        detAij = detAKL[i,j]
        #        invAij = invAKL[i,j]
        #        Rij = RIJ[i,j]
        #        matels = b_matel(n, Ai, Aj, invAij, detAij, detLi, detLj, Mass, Qmat,Rij)
        #        S[i,j] = S[i,j] + symc[k]*matels['skl'];
        #        T[i,j] = T[i,j] + symc[k]*matels['tkl'];
        #        V[i,j] = V[i,j] + symc[k]*matels['vkl'];
        
        S = S + symc[k]*SKL
        T = T + symc[k]*TKL
        V = V + symc[k]*VKL
        
        #print(bT)
        #print(T)
    
    H = T + V
    #dH = dT + dV
    
    # complete upper triangle of H and S
    for i in range(0,nb):
        for j in range(i+1,nb):
            H[j,i] = H[i,j]
            S[j,i] = S[i,j]
            #H[i,j] = H[j,i];
            #S[i,j] = S[j,i];
    th.set_printoptions(linewidth=120)        
    #print(V)
    cHc = c@H@c;
    cSc = c@S@c;
    eng = cHc/cSc;
    
    return eng           

In [533]:
def test_energyrc():
        
    n=3;
    nb=8;
    
    Mass = th.tensor([[0.5, 0.0, 0.0],
                     [0.0, 0.5, 0.0],
                     [0.0, 0.0, 0.5]], device=device, dtype=dtype);
    
    Charge = th.tensor([-3, 1, 1, -3, 1, -3], device=device, dtype=dtype);
    Charge = vech2L(Charge,n)
    
    # symmetry projection terms
    Sym = th.zeros((3,3,6), device=device, dtype=dtype)
    # (1)(2)(3)
    Sym[:,:,0] = th.tensor([[1,0,0],[0,1,0],[0,0,1]], device=device, dtype=dtype);
    # (12)
    Sym[:,:,1] = th.tensor([[0,1,0],[1,0,0],[0,0,1]], device=device, dtype=dtype);
    # (13)
    Sym[:,:,2] = th.tensor([[0,0,1],[0,1,0],[1,0,0]], device=device, dtype=dtype);
    # (23)
    Sym[:,:,3] = th.tensor([[1,0,0],[0,0,1],[0,1,0]], device=device, dtype=dtype);
    # (123)
    Sym[:,:,4] = th.tensor([[0,1,0],[0,0,1],[1,0,0]], device=device, dtype=dtype);
    # (132)
    Sym[:,:,5] = th.tensor([[0,0,1],[1,0,0],[0,1,0]], device=device, dtype=dtype);

    # coeff's
    symc = th.tensor([4.0,4.0,-2.0,-2.0,-2.0,-2.0], device=device, dtype=dtype);

    
    xvechL=th.tensor([
     1.6210e+00,
    -2.1504e-01,
     9.0755e-01,
     9.7866e-01,
    -2.8418e-01,
    -3.5286e+00,
    -3.3045e+00,
    -4.5036e+00,
    -3.2116e-01,
    -7.1901e-02,
     1.5167e+00,
    -8.4489e-01,
    -2.1377e-01,
    -3.6127e-03,
    -5.3774e-03,
    -2.1263e+00,
    -2.5191e-01,
     2.1235e+00,
    -2.1396e-01,
    -1.4084e-03,
    -1.0092e-02,
     4.5349e+00,
     9.4837e-03,
     1.1225e+00,
    -2.1315e-01,
     5.8451e-02,
    -4.9410e-03,
     5.0853e+00,
     7.3332e-01,
     5.0672e+00,
    -2.1589e-01,
    -6.8986e-03,
    -1.4310e-02,
     1.5979e+00,
     3.3946e-02,
    -8.7965e-01,
    -1.1121e+00,
    -2.1903e-03,
    -4.6925e-02,
     2.1457e-01,
     3.3045e-03,
     4.5120e+00,
    -2.1423e-01,
    -1.6493e-02,
    -2.3429e-03,
    -8.6715e-01,
    -6.7070e-02,
     1.5998e+00
     ], device=device, dtype=dtype, requires_grad=False)
    
    evec = th.tensor([
      -6.0460e-02,
       7.7708e-05,
       1.6152e+00,
       9.5443e-01,
       1.1771e-01,
       3.2196e+00,
       9.6344e-01,
       3.1398e+00
    ], device=device, dtype=dtype, requires_grad=False)
    
    #x1 = th.tensor(th.cat((xvechL,evec)), device=device, dtype=dtype, requires_grad=True)

    #energy = b_energyrc(x1,n,nb,Mass,Charge,Sym,symc)
    #print(energy)
    #return x1
    
    n=3;
    nb=256;
    #th.manual_seed(42)
    #x1 = th.randn(int(nb*n*(n+1)/2 + nb) , device=device, dtype=dtype, requires_grad=True)
    x1 = xrestart
    #print(x1)
    #energy, G = py_energyrc(x1,n,nb,Mass,Charge,Sym,symc)
    #energy = b_energyrc(x1,n,nb,Mass,Charge,Sym,symc)
    #print(energy)
    #return x1
    
    #optimizer = th.optim.LBFGS([x1])
    optimizer = th.optim.Adadelta([x1], lr=2.0)
    #optimizer = th.optim.Adam([x1], lr=0.5)
    
    scheduler = th.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', verbose=True,patience=2, factor=0.5)
    
    for i in range(1):
        optimizer.zero_grad()
        loss = b_energyrc(x1,n,nb,Mass,Charge,Sym,symc)
        loss.backward()
        #def closure():
        #    return b_energyrc(x1,n,nb,Mass,Charge,Sym,symc)
        optimizer.step()
        scheduler.step(loss)
        
        print('step: {} f: {} gradNorm: {}'.format(i, loss, th.norm(x1.grad)))
    
    return x1


In [534]:
start_time = time.time()
xrestart = test_energyrc()
print(" took {} seconds ".format(time.time() - start_time))
#cProfile.run('xrestart = test_energyrc()')

step: 0 f: -7.442151726305396 gradNorm: 0.18193230712624947
 took 164.13843297958374 seconds 


In [535]:
 th.save(xrestart, 'Libo-nb256-7.442.pt')

In [ ]:
xrestart = th.load('Libo-nb64-7.450.pt')